In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
url_1 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/14-1.csv'
url_2 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/14-2.csv'
url_3 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/14-3.csv'
url_4 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/14-4.csv'
url_5 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/14-5.csv'

data_1 = pd.read_csv(url_1)
data_2 = pd.read_csv(url_2)
data_3 = pd.read_csv(url_3)
data_4 = pd.read_csv(url_4)
data_5 = pd.read_csv(url_5)

data = data_1
data = data.append(data_2)
data = data.append(data_3)
data = data.append(data_4)
data = data.append(data_5)

data = data.reset_index()
data = data.drop(labels=['index'], axis=1)

### [1]

In [60]:
d1 = data[~(data.Class.isna()) | (data.Class.isna() & (data.Amount > 200))]

In [61]:
from sklearn.neighbors import KNeighborsClassifier

X_var = d1.columns[1:-2]
train = d1[~d1.Class.isna()]
test = d1[d1.Class.isna()]

knn = KNeighborsClassifier(n_neighbors=5).fit(train[X_var], train.Class)
pred = knn.predict_proba(test[X_var])
pred = pd.DataFrame(pred, columns=['neg', 'pos'])

In [62]:
pred = pred.assign(esti=pred.pos.apply(lambda x: 1 if x > 0 else 0))

In [63]:
test = test.reset_index()
test = test.assign(Class=pred.esti)
test = test.set_index(keys='index')

In [67]:
d1 = pd.concat([train, test])

In [72]:
print(f"{d1[d1.Class == 1].Amount.max() - d1[d1.Class == 1].Amount.min():.0f}")

25691


### [2]

In [79]:
from scipy.stats import ttest_ind

class1 = d1[d1.Class == 1].V2
class2 = d1[d1.Class == 0].V2

t_val, p_val = ttest_ind(class1, class2, equal_var=False)

print(f"{math.floor(abs(t_val)):.0f}")

22


### [3]

In [90]:
from sklearn.linear_model import LinearRegression

train0 = d1[d1.Class == 0]
train1 = d1[d1.Class == 1]

model0 = LinearRegression().fit(train0[X_var], train0.Amount)
model1 = LinearRegression().fit(train1[X_var], train1.Amount)

In [97]:
r_squared = model1.score(train1[X_var], train1.Amount)

In [98]:
print(f"{math.floor(r_squared * 100) / 100}")

0.93


### [4]

In [120]:
train = d1[d1.Time <= 100000]
X_var = train.columns[1:-1]

In [121]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(penalty='l2', C=100000, random_state=1234,
                           solver='newton-cg').fit(train[X_var], train.Class)

In [122]:
test = d1[d1.Time > 100000]

In [123]:
pred = model.predict_proba(test[X_var])
pred = pd.DataFrame(pred, columns=['neg', 'pos'])
pred = pred.assign(esti=pred.pos.apply(lambda x: 1 if x >= 0.4 else 0))

In [124]:
test = test.reset_index()
pred = pd.merge(pred, test.Class, left_index=True, right_index=True)
pred = pred.assign(accu=(pred.esti == pred.Class) * 1)
accuracy = pred.accu.sum() / pred.accu.count()

In [125]:
print(f"{math.floor(accuracy * 1000) / 1000}")

0.815
